### Sprint15 論文読解入門

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。


[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


https://arxiv.org/pdf/1506.01497.pdf

### 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。

### (1) 物体検出の分野にはどういった手法が存在したか。

オブジェクト提案方法には、スーパーピクセルのグループ化に基づく方法（選択的検索[4]、CPMC [22]、MCG [23]など）やスライディングウィンドウに基づく方法（例：ウィンドウのオブジェクト性[24]、EdgeBoxes [ 6]）。オブジェクト提案手法は、検出器とは独立した外部モジュールとして採用されました（選択的検索[4]オブジェクト検出器、RCNN [5]、高速R-CNN [2]など）

SPPnet [1]

Fast R-CNN [2]

今回の物は、2つのモジュールで構成されています。最初のモジュールは、領域を提案する深い完全たたみ込みネットワークで、2番目のモジュールは、提案された領域を使用するFast R-CNN検出器[2]

### (2) Fasterとあるが、どういった仕組みで高速化したのか。

最近注目されているニューラルネットワークの用語に「注意」メカニズムを使用して、RPNコンポーネントは統合ネットワークにどこを見るかを指示

これまでは、選択的検索[4]で、設計された低レベル機能に基づいてスーパーピクセルを貪欲にマージするので時間かかった。（1桁遅かった）

3 FASTER R-CNNから

### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

OverFeatは、1ステージのクラス固有の検出パイプラインであり、今回の手法は、クラスにとらわれない提案とクラス固有の検出で構成される2ステージのカスケードです。OverFeatでは、領域ごとの機能は、スケールピラミッド上の1つのアスペクト比のスライディングウィンドウから取得されます。
RPNでは、機能は正方形（3x3）のスライディングウィンドウからのものであり、異なるスケールとアスペクト比のアンカーに関連する提案を予測します。どちらの方法もスライディングウィンドウを使用しますが、領域提案タスクはFaster RCNNの最初の段階にすぎません。
カスケードの第2ステージでは、地域ごとの機能が、地域の機能をより忠実にカバーする提案ボックスから適応的にプールされます[1]、[2]。
これらの機能は、より正確な検出につながる

論文場所：P10 左辺り

One-Stage Detection vs. Two-Stage Proposal + Detection. The OverFeat paper [9] proposes a detection
method that uses regressors and classifiers on sliding
windows over convolutional feature maps. OverFeat
is a one-stage, class-specific detection pipeline, and ours
is a two-stage cascade consisting of class-agnostic proposals and class-specific detections. In OverFeat, the
region-wise features come from a sliding window of
one aspect ratio over a scale pyramid. These features
are used to simultaneously determine the location and
category of objects. In RPN, the features are from
square (3×3) sliding windows and predict proposals
relative to anchors with different scales and aspect
ratios. Though both methods use sliding windows, the
region proposal task is only the first stage of Faster RCNN—the downstream Fast R-CNN detector attends
to the proposals to refine them. In the second stage of
our cascade, the region-wise features are adaptively
pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions. We believe
these features lead to more accurate detections.

### (4) RPNとは何か。

検出ネットワークとフルイメージのたたみ込み機能を共有するリージョン提案ネットワーク（RPN）

完全にたたみ込みネットワークであり、各位置でオブジェクトの境界とオブジェクトネススコアを同時に予測

（任意のサイズの）画像を入力として受け取り、それぞれがオブジェクトネススコアを持つ一連の長方形のオブジェクト提案を出力

エンドツーエンドでトレーニングされ、高速R-CNNが検出に使用する高品質の地域提案を生成

Fast RCNNのような領域ベースの検出器で使用される畳み込み特徴マップは、領域提案の生成にも使用できることです。これらのたたみ込み機能に加えて、通常のグリッドの各位置で領域の境界とオブジェクトネススコアを同時に後退させるいくつかの追加のたたみ込み層を追加して、RPNを構築します。したがって、RPNは一種の完全たたみ込みネットワーク（FCN）[7]であり、特に検出提案を生成するタスクのためにエンドツーエンドでトレーニングできます

### (5) RoIプーリングとは何か。

入力を固定長に変換するアルゴリズムを適用し、入力特徴マップのサイズにかかわらず固定長の長さに変換する。

領域検出された箇所に対し最大プーリングを行う。処理後のサイズは任意の大きさに決めることができる。

例えば、領域検出された箇所が複数存在し大きさがバラバラであっても、指定したサイズに出力される。

各領域が同じサイズになるため、どの領域も最後の全結合に通すことができる。


[1]Spatial pyramid pooling in deep convolutional networks for visual recognition, 2.2 The Spatial Pyramid Pooling Layer、
および [2]Fast R-CNN, 2.1. The RoI pooling layerから

### (6) Anchorのサイズはどうするのが適切か。

ボックス領域が1282、2562、および5122ピクセルの3つのスケールと、1：1、1：2、および2：1の3つのアスペクト比を使用
各位置でアンカーを1つだけ使用する場合、mAPは3〜4％のかなりのマージンで低下します。 3スケール（1アスペクト比）または3アスペクト比（1スケール）を使用すると、mAPは高くなります。これは、回帰参照として複数のサイズのアンカーを使用することが効果的なソリューションであることを示しています。 1つのアスペクト比（69.8％）の3つのスケールを使用することは、このデータセットで3つのアスペクト比の3つのスケールを使用することと同じです。つまり、スケールとアスペクト比は、検出精度のために絡み合っていない次元ではありません。しかし、システムに柔軟性を持たせるために、これらの2つの次元を設計に採用しています。

論文場所：P9 右上辺り

If using just one anchor at each position, the mAP
drops by a considerable margin of 3-4%. The mAP
is higher if using 3 scales (with 1 aspect ratio) or 3
aspect ratios (with 1 scale), demonstrating that using
anchors of multiple sizes as the regression references
is an effective solution. Using just 3 scales with 1
aspect ratio (69.8%) is as good as using 3 scales with
3 aspect ratios on this dataset, suggesting that scales
and aspect ratios are not disentangled dimensions for
the detection accuracy. But we still adopt these two
dimensions in our designs to keep our system flexible.

### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

データ：PASCAL VOC 2007検出ベンチマーク[11]、20のオブジェクトカテゴリにわたる約5kのtrainval画像と5kのテスト画像で構成、

mageNetの事前トレーニング済みネットワークでは、5つの畳み込み層と3つの完全に接続された層を持つZFネット[32]の「高速」バージョンと、

13の畳み込み層と3を持つパブリックVGG-16モデル7 [3]を使用

指標値：検出平均の平均精度（mAP）

論文場所：7P 左中央辺り

4.1 Experiments on PASCAL VOC
We comprehensively evaluate our method on the
PASCAL VOC 2007 detection benchmark [11]. This
dataset consists of about 5k trainval images and 5k
test images over 20 object categories. We also provide
results on the PASCAL VOC 2012 benchmark for a
few models. For the ImageNet pre-trained network,
we use the “fast” version of ZF net [32] that has
5 convolutional layers and 3 fully-connected layers,
and the public VGG-16 model7
[3] that has 13 convolutional layers and 3 fully-connected layers. We
primarily evaluate detection mean Average Precision
(mAP), because this is the actual metric for object
detection (rather than focusing on object proposal
proxy metrics).

#### (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。